# Libraries

In [32]:
import math
import pandas as pd
import inflection 

### Helper Functions

### Loading Data

Firstly, we are going to check all datasets out. We must verify what kind of fields there are in every dataset. 

In [6]:
data_raw_train = pd.read_csv("datasets/train.csv", low_memory=False)
data_raw_train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [7]:
data_raw_test = pd.read_csv("datasets/test.csv", low_memory=False)
data_raw_test.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0


In [8]:
data_raw_store = pd.read_csv("datasets/store.csv", low_memory=False)
data_raw_store.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [9]:
data_raw_sample = pd.read_csv("datasets/sample_submission.csv", low_memory=False)
data_raw_sample.head()

,Id,Sales
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


Based on the data above, we are going to merge the datasets "train" and "store". They are the ones with the relevant data we need to perform the forecast. 

In [10]:
data_raw = pd.merge(data_raw_train, data_raw_store, how="left", on="Store")
data_raw.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,5,2015-07-31,4822,559,1,1,0,1,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


# 1. Data Description

In [11]:
df1 = data_raw.copy()

## 1.1. Rename Columns

We are going to rename columns from camel case to snake case.

In [12]:
df1.columns

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval'],
      dtype='object')

In [14]:
cols_old = ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday', 
            'StoreType', 'Assortment', 'CompetitionDistance', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear',
            'Promo2', 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval']

snake_case = lambda x: inflection.underscore(x)

cols_new = list(map(snake_case, cols_old))

#rename
df1.columns = cols_new

In [15]:
df1.columns

Index(['store', 'day_of_week', 'date', 'sales', 'customers', 'open', 'promo',
       'state_holiday', 'school_holiday', 'store_type', 'assortment',
       'competition_distance', 'competition_open_since_month',
       'competition_open_since_year', 'promo2', 'promo2_since_week',
       'promo2_since_year', 'promo_interval'],
      dtype='object')

## 1.2. Data Dimensions

In [19]:
print("Number of rows: {}".format(df1.shape[0]))
print("Number of cols: {}".format(df1.shape[1]))

Number of rows: 1017209
Number of cols: 18


## 1.3. Data Types

In [20]:
df1.dtypes

store                             int64
day_of_week                       int64
date                             object
sales                             int64
customers                         int64
open                              int64
promo                             int64
state_holiday                    object
school_holiday                    int64
store_type                       object
assortment                       object
competition_distance            float64
competition_open_since_month    float64
competition_open_since_year     float64
promo2                            int64
promo2_since_week               float64
promo2_since_year               float64
promo_interval                   object
dtype: object

In [21]:
df1.sample()

,store,day_of_week,date,sales,customers,open,promo,state_holiday,school_holiday,store_type,assortment,competition_distance,competition_open_since_month,competition_open_since_year,promo2,promo2_since_week,promo2_since_year,promo_interval
526376,882,1,2014-03-17,7518,960,1,1,0,0,a,a,30.0,4.0,2013.0,0,NaN,NaN,NaN


In [28]:
df1["state_holiday"].unique()

array(['0', 'a', 'b', 'c'], dtype=object)

In [38]:
df1["date"] = pd.to_datetime( df1["date"] )

In [39]:
df1.dtypes

store                                    int64
day_of_week                              int64
date                            datetime64[ns]
sales                                    int64
customers                                int64
open                                     int64
promo                                    int64
state_holiday                           object
school_holiday                           int64
store_type                              object
assortment                              object
competition_distance                   float64
competition_open_since_month           float64
competition_open_since_year            float64
promo2                                   int64
promo2_since_week                      float64
promo2_since_year                      float64
promo_interval                          object
dtype: object

## 1.4. Check NA/Missing values

In [29]:
df1.isna().sum()

store                                0
day_of_week                          0
date                                 0
sales                                0
customers                            0
open                                 0
promo                                0
state_holiday                        0
school_holiday                       0
store_type                           0
assortment                           0
competition_distance              2642
competition_open_since_month    323348
competition_open_since_year     323348
promo2                               0
promo2_since_week               508031
promo2_since_year               508031
promo_interval                  508031
dtype: int64

In [30]:
df1.isnull().sum()

store                                0
day_of_week                          0
date                                 0
sales                                0
customers                            0
open                                 0
promo                                0
state_holiday                        0
school_holiday                       0
store_type                           0
assortment                           0
competition_distance              2642
competition_open_since_month    323348
competition_open_since_year     323348
promo2                               0
promo2_since_week               508031
promo2_since_year               508031
promo_interval                  508031
dtype: int64

## 1.5. Fillout NA/Missing values

We have 03 different types of treating NA values:

**1.** Deleting all of them. It is easy and fast, but it results in a big loss. 

**2.** We may also use Machine Learning algorithms to fill them based on the column characteristics. e.g.: using median, mean or even clustering. 

**3.** We can fill them based on business decisions. That's the one we are going to use for now.   

#### competition_distance

This column means the "distance in meters to the nearest competitor store". We may assume that NA values are the ones greater than the maximum distance between the stores. 

In [31]:
df1["competition_distance"].max()

75860.0

As the maximum distance between competitors store is 75860.0 m, we may replace NA values with 200000.0. This replacement means, in other words, that the store has no competitor - based on the competition_distance field.  

In [33]:
df1["competition_distance"] = df1["competition_distance"].apply(lambda x: 200000.0 if math.isnan(x) else x)

#### competition_open_since_month

This field gives the approximate month of the time the nearest competitor was opened. We are going to use the month from the date field where there are NA values in the column competition_since_open_month. 

In [35]:
df1["competition_open_since_month"].min()

1.0

In [36]:
df1["competition_open_since_month"].unique()

array([ 9., 11., 12.,  4., 10.,  8., nan,  3.,  6.,  5.,  1.,  2.,  7.])

We are going to replace NA values with the month of the date of the row. 

In [44]:
df1["competition_open_since_month"] = df1.apply(lambda x: 
                                                x["date"].month if math.isnan(x["competition_open_since_month"]) 
                                                else x["competition_open_since_month"], axis=1)

#### competition_open_since_year

The procedure is very similar to the last treatment. However, we are going to use year instead of month. 

In [46]:
df1["competition_open_since_year"].unique()

array([2008., 2007., 2006., 2009., 2015., 2013., 2014., 2000., 2011.,
         nan, 2010., 2005., 1999., 2003., 2012., 2004., 2002., 1961.,
       1995., 2001., 1990., 1994., 1900., 1998.])

We are going to replace NA values with the month of the date of the row.

In [47]:
df1["competition_open_since_year"] = df1.apply(lambda x: 
                                                x["date"].year if math.isnan(x["competition_open_since_year"]) 
                                                else x["competition_open_since_year"], axis=1)

#### promo2_since_week

Promo2 is a continuing and consecutive promotion for some stores. promo2_since_week describes the week when the store started participating in Promo2. The treatment is similar to the last one above, yet using week as time period. 

In [49]:
df1["promo2_since_week"] = df1.apply(lambda x: 
                                    x["date"].week if math.isnan(x["promo2_since_week"]) 
                                    else x["promo2_since_week"], axis=1)

In [50]:
df1.isna().sum()

store                                0
day_of_week                          0
date                                 0
sales                                0
customers                            0
open                                 0
promo                                0
state_holiday                        0
school_holiday                       0
store_type                           0
assortment                           0
competition_distance                 0
competition_open_since_month         0
competition_open_since_year          0
promo2                               0
promo2_since_week                    0
promo2_since_year               508031
promo_interval                  508031
dtype: int64

#### promo2_since_year

Similar to the treatment above. Yet, using year as time period. 

In [52]:
df1["promo2_since_year"] = df1.apply(lambda x: 
                                    x["date"].year if math.isnan(x["promo2_since_year"]) 
                                    else x["promo2_since_year"], axis=1)

In [53]:
df1.isna().sum()

store                                0
day_of_week                          0
date                                 0
sales                                0
customers                            0
open                                 0
promo                                0
state_holiday                        0
school_holiday                       0
store_type                           0
assortment                           0
competition_distance                 0
competition_open_since_month         0
competition_open_since_year          0
promo2                               0
promo2_since_week                    0
promo2_since_year                    0
promo_interval                  508031
dtype: int64

#### promo_interval

This field describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store.

For this treatment, we created a new column named "is_promo" where: 

1. 0 means there is no promo2 because promo_interval is 0 (previous NA).
2. 1 if the month from date field is in promo_interval field. 

In [54]:
month_map = {1: "Jan", 2: "Feb", 3: "Mar", 4: "Apr", 5: "May", 6: "Jun", 7: "Jul", 8: "Aug", 9: "Sep", 10: "Oct", 11: "Nov", 12: "Dec", }

df1["promo_interval"].fillna(0, inplace=True)

df1["month_map"] = df1["date"].dt.month.map(month_map) 

df1["is_promo"] = df1[["promo_interval", "month_map"]].apply(lambda x: 0 if x["promo_interval"] == 0 
                                                              else 1 if x["month_map"] in x["promo_interval"].split(",") else 0, axis=1)

In [55]:
df1.sample(5)

,store,day_of_week,date,sales,customers,open,promo,state_holiday,school_holiday,store_type,assortment,competition_distance,competition_open_since_month,competition_open_since_year,promo2,promo2_since_week,promo2_since_year,promo_interval,month_map,is_promo
843822,553,3,2013-06-05,5303,461,1,1,0,0,c,a,3040.0,9.0,2002.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",Jun,0
481485,591,6,2014-04-26,3919,335,1,0,0,0,a,c,3340.0,4.0,2014.0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",Apr,0
453616,597,3,2014-05-21,4781,716,1,1,0,0,a,a,150.0,9.0,2008.0,1,1.0,2012.0,"Mar,Jun,Sept,Dec",May,0
912580,181,4,2013-04-04,5864,670,1,0,0,1,a,a,12610.0,3.0,2013.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",Apr,1
749583,1089,4,2013-08-29,10260,972,1,1,0,1,d,a,5220.0,5.0,2009.0,0,35.0,2013.0,0,Aug,0


In [56]:
df1.isna().sum()

store                           0
day_of_week                     0
date                            0
sales                           0
customers                       0
open                            0
promo                           0
state_holiday                   0
school_holiday                  0
store_type                      0
assortment                      0
competition_distance            0
competition_open_since_month    0
competition_open_since_year     0
promo2                          0
promo2_since_week               0
promo2_since_year               0
promo_interval                  0
month_map                       0
is_promo                        0
dtype: int64

## 1.6. Change Data Types

Since we did many operations with the columns in the last section, it is quite important to check their types and change them if necessary. 

In [57]:
df1.dtypes

store                                    int64
day_of_week                              int64
date                            datetime64[ns]
sales                                    int64
customers                                int64
open                                     int64
promo                                    int64
state_holiday                           object
school_holiday                           int64
store_type                              object
assortment                              object
competition_distance                   float64
competition_open_since_month           float64
competition_open_since_year            float64
promo2                                   int64
promo2_since_week                      float64
promo2_since_year                      float64
promo_interval                          object
month_map                               object
is_promo                                 int64
dtype: object

In [58]:
df1.sample(3)

,store,day_of_week,date,sales,customers,open,promo,state_holiday,school_holiday,store_type,assortment,competition_distance,competition_open_since_month,competition_open_since_year,promo2,promo2_since_week,promo2_since_year,promo_interval,month_map,is_promo
95779,1005,4,2015-05-07,6437,906,1,1,0,0,a,a,6480.0,5.0,2015.0,0,19.0,2015.0,0,May,0
146180,116,7,2015-03-22,0,0,0,0,0,0,a,a,680.0,4.0,2013.0,0,12.0,2015.0,0,Mar,0
972748,139,6,2013-02-09,3806,323,1,0,0,0,a,a,1700.0,1.0,2008.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",Feb,0


In [61]:
df1["sales"] = df1["sales"].astype(float)
df1["competition_open_since_month"] = df1["competition_open_since_month"].astype(int)
df1["competition_open_since_year"] = df1["competition_open_since_year"].astype(int)
df1["promo2_since_week"] = df1["promo2_since_week"].astype(int)
df1["promo2_since_year"] = df1["promo2_since_year"].astype(int)

In [62]:
df1.dtypes

store                                    int64
day_of_week                              int64
date                            datetime64[ns]
sales                                  float64
customers                                int64
open                                     int64
promo                                    int64
state_holiday                           object
school_holiday                           int64
store_type                              object
assortment                              object
competition_distance                   float64
competition_open_since_month             int64
competition_open_since_year              int64
promo2                                   int64
promo2_since_week                        int64
promo2_since_year                        int64
promo_interval                          object
month_map                               object
is_promo                                 int64
dtype: object

In [63]:
df1.sample(3)

,store,day_of_week,date,sales,customers,open,promo,state_holiday,school_holiday,store_type,assortment,competition_distance,competition_open_since_month,competition_open_since_year,promo2,promo2_since_week,promo2_since_year,promo_interval,month_map,is_promo
175572,518,2,2015-02-24,3961.0,393,1,0,0,0,d,c,3330.0,3,2015,1,13,2010,"Jan,Apr,Jul,Oct",Feb,0
552591,337,5,2014-02-21,5549.0,411,1,1,0,0,d,c,10600.0,7,2005,1,45,2014,"Feb,May,Aug,Nov",Feb,1
207266,992,2,2015-01-27,7267.0,798,1,1,0,0,a,a,2480.0,7,1990,0,5,2015,0,Jan,0
